In [3]:
import cv2
import pandas as pd
import numpy as np
import os

# Funkcija za prikaz i brojanje sa prelaskom vertikalne linije
def count_and_visualize_buzzy_beetles(video_path):
    cap = cv2.VideoCapture(video_path)
    count = 0
    tracked_objects = []  # Lista za praćenje objekata (x, y)
    min_contour_area = 1000  # Minimalna veličina konture da bi se smatrala validnim objektom
    max_contour_area = 2000  # Maksimalna veličina konture
    distance_threshold = 40  # Maksimalna udaljenost između detekcija za isti objekat

    print(f"Processing {video_path}...")

    # Dimenzije linije za praćenje (x = vertikalna linija)
    ret, first_frame = cap.read()
    if not ret:
        print("Error reading video")
        return count
    h, w, _ = first_frame.shape
    track_line = 450  # Vertikalna linija u sredini frejma

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Resetovanje na početak videa

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Kropovanje na srednji kvadrat
        h, w, _ = frame.shape
        size = min(h, w)
        start_x = (w - size) // 2
        start_y = (h - size) // 2 + 200
        frame = frame[start_y:start_y+size - 100, start_x:start_x+size]

        # Konverzija u HSV i maskiranje tamno plave boje
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lower_blue = np.array([85, 80, 45])  # Donja granica za tamno plavu
        upper_blue = np.array([140, 255, 255])  # Gornja granica za tamno plavu
        mask = cv2.inRange(hsv_frame, lower_blue, upper_blue) # binarna maska gde su pikseli koji upadaju u opseg hsv 1 a ostalo 0
        
        # Pronalaženje kontura
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        objects_detected = False  # Flag za detekciju objekata
        for contour in contours:
            area = cv2.contourArea(contour)
            if min_contour_area <= area <= max_contour_area:
                x, y, w, h = cv2.boundingRect(contour)
                center_x = x + w // 2
                center_y = y + h // 2

                # Provera da li je objekat već praćen
                is_new_object = True
                for (prev_x, prev_y) in tracked_objects:
                    if np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2) < distance_threshold:
                        is_new_object = False
                        break

                # Ako je novi objekat, dodaj ga u praćenje
                if is_new_object:
                    tracked_objects.append((center_x, center_y))
                    # Provera prelaska vertikalne linije
                    if center_x > track_line:
                        count += 1

                        # Prikazivanje frejma na kojem se broji
                        cv2.imshow("Frame with Count", frame)
                        cv2.waitKey(1)  # Pauza za prikaz

                objects_detected = True
                # Vizualizacija detekcije: pravougaonik
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Vizualizacija linije za praćenje
        cv2.line(frame, (track_line, 0), (track_line, h), (255, 0, 0), 2)  # Vertikalna linija

    cap.release()
    cv2.destroyAllWindows()
    return count

# Funkcija za obradu svih videa i evaluaciju
def process_videos_with_visualization(video_folder, ground_truth_path):
    ground_truth = pd.read_csv(ground_truth_path)
    videos = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.endswith('.mp4')]

    predicted_counts = []
    for video_path in sorted(videos):
        video_name = os.path.basename(video_path)
        count = count_and_visualize_buzzy_beetles(video_path)
        predicted_counts.append({'video': video_name, 'predicted_count': count})

    predicted_df = pd.DataFrame(predicted_counts)
    result_df = pd.merge(ground_truth, predicted_df, on='video')
    result_df['error'] = abs(result_df['count'] - result_df['predicted_count'])
    mae = result_df['error'].mean()

    # Prikaz rezultata
    print(result_df)
    print(f"Mean Absolute Error (MAE): {mae}")
    return result_df, mae

# Pokretanje analize sa vizualizacijom
video_folder = 'data'  # Folder sa video fajlovima
ground_truth_path = 'data/buzzy_beetle_count.csv'

result_df, mae = process_videos_with_visualization(video_folder, ground_truth_path)


Processing data\video_1.mp4...
Processing data\video_10.mp4...
Processing data\video_2.mp4...
Processing data\video_3.mp4...
Processing data\video_4.mp4...
Processing data\video_5.mp4...
Processing data\video_6.mp4...
Processing data\video_7.mp4...
Processing data\video_8.mp4...
Processing data\video_9.mp4...
          video  count  predicted_count  error
0   video_1.mp4      2                2      0
1   video_2.mp4      1                1      0
2   video_3.mp4      2                2      0
3   video_4.mp4      3                4      1
4   video_5.mp4      1                1      0
5   video_6.mp4      1                1      0
6   video_7.mp4      1                1      0
7   video_8.mp4      1                1      0
8   video_9.mp4      2                0      2
9  video_10.mp4      2                2      0
Mean Absolute Error (MAE): 0.3
